*Required* python Library

In [ ]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics,svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Downloading the Dataset from Kaggel

Load the file and create a Pandas DataFrame

In [ ]:
raw_df = pd.read_csv("/kaggle/input/apaddos-dataset/APA-DDoS-Dataset/APA-DDoS-Dataset.csv", delimiter=',')
raw_df.head()

List of the Dataset classifiers of ML model Building

In [ ]:
raw_df['Label'].unique()

In [ ]:
raw_df = raw_df.assign(
balance_Label = lambda dataframe: dataframe['Label'].map(lambda val: 'DDoS' if val == 'DDoS-PSH-ACK' or val == 'DDoS-ACK' else 'Benign') 
)
raw_df.head()

Balance Distribuation check of the dataset classifiers

In [ ]:

def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

Preprocessing of the Dataset for ML Model 

In [ ]:
clean_df=handle_non_numerical_data(raw_df)

In [ ]:
clean_df.head()

In [ ]:
sns.histplot(data=raw_df, x='balance_Label', hue='balance_Label', multiple='stack')
plt.show()

# Model Traning
---


1. ML Models  
2. Perfomance Report  
3. Feature Importance


spliting the data for the Traning in to 70/30 (Traning/test)

In [ ]:
feature_column = ['ip.src','ip.dst','tcp.srcport','tcp.dstport','ip.proto','tcp.flags.syn','tcp.flags.reset','tcp.flags.push','tcp.flags.ack','ip.flags.mf','ip.flags.df','ip.flags.rb','tcp.seq','tcp.ack','frame.time','Packets','Bytes','Tx Packets','Tx Bytes','Rx Packets','Rx Bytes']
X = clean_df[feature_column] 
Y = clean_df['balance_Label']
print(X.shape)
print(Y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42) 

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

## **List of models**


### 1.   Decision Tree
Decision Trees are a type of Supervised Machine Learning (you explain what the input is and what the corresponding output is in the training data) in which the data is continuously split based on a specific parameter.
### 2.   KNN : k-nearest neighbors 
KNN is an abbreviation for "K-Nearest Neighbour." It is a machine learning algorithm that is supervised. The algorithm can solve classification and regression problem statements. The symbol 'K' represents the number of nearest neighbors to a new unknown variable that must be predicted or classified.
### 3. Random Forest
Random forests, also known as random decision forests, are an ensemble learning method for classification, regression, and other tasks that works by constructing a large number of decision trees during training. For classification tasks, the random forest output is the class chosen by the majority of trees.
### 4. SVM : Support vector machine
Support-vector machines are supervised learning models that analyze data for classification and regression analysis using learning algorithms.
### 5. Neural Network
Neural networks, also known as artificial neural networks (ANNs) or simulated neural networks (SNNs), are a subset of machine learning that form the foundation of deep learning algorithms. Their name and structure are inspired by the human brain, and they mimic the way biological neurons communicate with one another.
### 6. GBT : Gradient boosting tree (*Advance ML Model*)
Gradient boosting is a machine learning technique that is commonly used in regression and classification tasks. It returns a prediction model in the form of an ensemble of weak prediction models, usually decision trees.



Temp List to store all the requied Model Data

In [ ]:
master_models=[] #[model_name,ml_model]
master_preduction=[] #[model_name,model_fit]
master_model_result=[] #[model_name,Accuracy, Precision, Detection  rate, False positive rate]

In [ ]:
k=3 #Hyper paramaters 
master_models.insert(-1,["Decision Tree",DecisionTreeClassifier()])
master_models.insert(-1,["KNeighbors",KNeighborsClassifier(n_neighbors=k)])
master_models.insert(-1,["RandomForest",RandomForestClassifier()])
master_models.insert(-1,["Support vector machine",svm.SVC(kernel='linear')])
master_models.insert(-1,["Neural Network",MLPClassifier(hidden_layer_sizes=(21,21,21), activation='relu', solver='adam', max_iter=500)])
master_models.insert(-1,["Gradient boosting",GradientBoostingClassifier(random_state=0)])

Model Building

In [ ]:
for name,model in master_models:
    print("Building model : ",name)
    model.fit(X_train,y_train)
    master_preduction.insert(-1,[name,model.predict(X_test)])

## ML Model Result 

In [ ]:
def detection_rate_score(cm):
    # tp/(tp+fn)
    return cm[0][0]/(cm[0][0]+cm[1][0])
def false_positive_score(cm):
    # fp/(fp+tn)
    return cm[0][1]/(cm[0][1]+cm[1][1])

In [ ]:
for name,predict in master_preduction:
    tmp_cm = confusion_matrix(y_test,predict)
    #Detection  rate
    #False positive rate
    master_model_result.insert(-1,[name,accuracy_score(y_test,predict),precision_score(y_test,predict),detection_rate_score(tmp_cm),false_positive_score(tmp_cm)])
    tmp_cm=0

Model Comparison DataFrame

In [ ]:
result_df = pd.DataFrame(master_model_result,columns=["model_name","Accuracy", "Precision", "Detection  rate", "False positive rate"])
display(result_df)

In [ ]:
cm = confusion_matrix(y_test, predict)

# Create a heatmap using Seaborn
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()